In [2]:
from tkinter import *
import pandas as pd
import random
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import neighbors
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

root = Tk()
root.title("AI-Based Overvoltage Control Parameter Prediction")
root.geometry("720x445")

fr_title = Frame(root)

lab = Label(fr_title, text="Enter known parameters and click \'Confirm\' to get the best set of parameters.")
lab.grid(row=0, column=0)
fr_title.grid(row=0, column=0, pady=20, columnspan=2)

fr_input = Frame(root)
fr_output = Frame(root)

names=['Voltage Threshold', 'J-weight', 'Slope', 'Stop rpm Threshold', 'P', 'I']
names_o=['Voltage', 'multiple', 'k', 'rpm', 'P', 'I']
border = [[710, 780], [16, 20], [5, 5], [60, 70], [5, 50], [0.01, 5.0119]]
algo_names = ['Decision Tree','Random Forest','SVM','kNN']

def Prediction(alg, quest):
    # 1. 加载CSV数据
    data = pd.read_csv('cleaned_file.csv')

    # 2. 假设特征在data的前几列，标签在最后一列
    X = data.iloc[:, :-1].values  # 输入特征
    y = data.iloc[:, -1].values   # 输出标签

    # 3. 数据分割为训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    if alg == 2:
        n_neighbors = 5
        knn = neighbors.KNeighborsRegressor(n_neighbors, weights="distance")
        knn_regressor = knn.fit(X_train, y_train)
        return knn_regressor.predict(quest)
    elif alg == 3:
        regressor = SVR(kernel='rbf', C=1, epsilon=0.01, gamma='auto')
        regressor.fit(X_train, y_train)
        return regressor.predict(quest)
    elif alg == 0:
        regressor = DecisionTreeRegressor(max_depth = 17, min_samples_leaf = 2, min_samples_split = 19) 
        regressor.fit(X_train, y_train)
        return regressor.predict(quest)
    elif alg == 1:
        random_forest_regressor = RandomForestRegressor(n_estimators=100, max_depth=17, max_features=5, random_state=0)
        random_forest_regressor.fit(X_train, y_train)
        return random_forest_regressor.predict(quest)
    else:
        return None

def callback():
    if not(valid_all()):
        return
    data = {
        'Voltage' : [],
        'multiple' : [],
        'k' : [],
        'rpm' : [],
        'P' : [],
        'I' : []
    }
    to_pred = []
    for i in range(6):
        if var[i].get() == "0":
            if names_o[i] == 'k':
                var[i].set('5')
            else:
                to_pred.append(i)
        #print(var[i].get())
    #print(to_pred)
    random_amount = 10000
    for i in range(random_amount):
        for j in range(6):
            if j in to_pred:
                data[names_o[j]].append(random.uniform(border[j][0],border[j][1]))
            else:
                data[names_o[j]].append(float(var[j].get()))
    quest = pd.DataFrame(data)

    pred = Prediction(algorithm.get(),quest)
    # pred = random_forest_regressor.predict(quest)
    a = np.where(pred == min(pred))
    #print(a)
    #print(a[0])
    res = []
    for i in range(6):
        res.append(quest[names_o[i]][a[0]].iloc[0])
    res.append(min(pred))
    
    for i in range(6):
        lab_ow[i] = Label(fr_output, text=names[i]+" : ")
        lab_ow[i].grid(row=i+1, column=0, sticky='e', pady=5)
        lab_oe[i] = Label(fr_output, text='{:.3f}'.format(res[i]))
        lab_oe[i].grid(row=i+1, column=1, sticky='w', pady=5)
    lab_end = Label(fr_output, text="And the stop duration is "+'{:.5f}'.format(min(pred))+" second(s).")
    lab_end.grid(row=7, column=0, sticky='w', pady=10)

def valid_all():
    flag = 1
    for i in range(6):
        try:
            float(var[i].get())
        except ValueError:
            flag = False
    if flag:
        #print("Start predicting.")
        lab_ill.grid_forget()
        return 1
    else:
        #print("Illegal input!")
        lab_ill.grid(row=10, column=0, pady=5)
        return 0
    
var = [StringVar() for i in range(6)]
lab = [None for i in range(6)]
lab_ow = [None for i in range(6)]
lab_oe = [None for i in range(6)]
ent = [None for i in range(6)]
for i in range(6):
    var[i].set("0")
lab_ill = Label(fr_input, text="Illegal input!")
for i in range(6):
    lab[i] = Label(fr_input, text=names[i]+" : ")
    lab[i].grid(row=i, column=0, sticky='e', pady=5)
    ent[i] = Entry(fr_input, validate='focusout', textvariable=var[i])
    ent[i].grid(row=i, column=1, sticky='w', pady=5)

lab_algo = Label(fr_input, text="Choose Algorithm:")
lab_algo.grid(row=6, column=0, pady=5)
algorithm = IntVar()

algo_choice = [None for i in range(4)]
for i in range(4):
    algo_choice[i] = Radiobutton(fr_input, variable=algorithm, text=algo_names[i], value=i)
    algo_choice[i].grid(row=7+i, column=0, pady=5)
algo_choice[0].grid(row=7, column=0, sticky='w', pady=5)
algo_choice[2].grid(row=7, column=1, sticky='w', pady=5)
algo_choice[1].grid(row=8, column=0, sticky='w', pady=5)
algo_choice[3].grid(row=8, column=1, sticky='w', pady=5)
    

button = Button(fr_input, text='Confirm', command=callback, relief='groove')
button.grid(row=9, column=0, columnspan=2, pady=10)

lab_intro = Label(fr_output, text="The best set of parameters is:")
lab_intro.grid(row=0, column=0, columnspan=2, pady=10, sticky='nw')


fr_input.grid(row=1, column=0, sticky='nw', padx=32)
fr_output.grid(row=1, column=1, sticky='nw', padx=32)



mainloop()


C:\Users\World\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
